In [1]:
import pandas as pd
import numpy as np
import time
import pickle

from itertools import chain 

In [2]:
%%time
statements_full_df = pd.read_csv('KDWD/statements.csv', dtype='int32')
display(statements_full_df.dtypes)
statements_full_df

source_item_id      int32
edge_property_id    int32
target_item_id      int32
dtype: object

CPU times: user 28 s, sys: 1.42 s, total: 29.5 s
Wall time: 29.5 s


,source_item_id,edge_property_id,target_item_id
0,1,31,36906466
1,1,279,3695190
2,1,398,497745
3,1,398,1133705
4,1,398,1139177
...,...,...,...
141206848,77257484,59,9286
141206849,77257491,31,318
141206850,77257491,59,9286
141206851,77257493,31,318


In [3]:
property_df = pd.read_csv('data/property_details.csv')
property_df.priority = property_df.priority.fillna(4).astype(int)
display(property_df)
np.unique(property_df.priority)

,priority,property_id,en_label,en_description,popularity,aliases,aliases_count
0,1,31,instance of,that class of which this subject is a particul...,26016076,"['is a', 'is an', 'has class', 'has type', 'is...",24.0
1,1,279,subclass of,all instances of these items are instances of ...,1731815,"['rdfs:subClassOf', 'hyponym of', 'has supercl...",19.0
2,1,460,said to be the same as,"this item is said to be the same as that item,...",78860,"['same as', 'disputed equivalence', 'the same ...",15.0
3,1,734,family name,part of full name of person,1692849,"['last name', 'surname']",2.0
4,1,155,follows,immediately prior item in a series of which th...,289262,"['succeeds to', 'previous is', 'before was', '...",11.0
...,...,...,...,...,...,...,...
1213,4,5439,research measurement,a measurement of an experimentally observed va...,1,NaN,NaN
1214,4,7174,school class,(qualifier) class of a given year/time period ...,1,"['school cohort', 'cohort', 'class']",3.0
1215,4,2443,stage reached,ultimate point in an event or competition reac...,1,"['eliminated at', 'round reached', 'point achi...",3.0
1216,4,1310,statement disputed by,entity that disputes a given statement,1,"['disputed by', 'rejected by', 'opposed by']",3.0


array([ 1,  2,  3,  4,  8,  9, 10])

In [4]:
property_priority_dict = property_df.set_index('property_id').priority.to_dict()
property_priority_dict[31]

1

In [5]:
%%time
statements_full_df['priority'] = [property_priority_dict[i] for i in statements_full_df.edge_property_id]
statements_full_df

CPU times: user 1min 10s, sys: 1.6 s, total: 1min 11s
Wall time: 1min 11s


,source_item_id,edge_property_id,target_item_id,priority
0,1,31,36906466,1
1,1,279,3695190,1
2,1,398,497745,4
3,1,398,1133705,4
4,1,398,1139177,4
...,...,...,...,...
141206848,77257484,59,9286,8
141206849,77257491,31,318,1
141206850,77257491,59,9286,8
141206851,77257493,31,318,1


In [6]:
%%time
statements_filtered_df = statements_full_df[statements_full_df.priority <= 4].copy()
display(statements_filtered_df)

weight_mapping = {1:8, 2:7, 3:6, 4:3}
statements_filtered_df['weight'] = [weight_mapping.get(i, -1) for i in statements_filtered_df.priority]
statements_filtered_df = statements_filtered_df.groupby(['source_item_id', 'target_item_id'])['weight'].sum().to_frame().reset_index()
statements_filtered_df

,source_item_id,edge_property_id,target_item_id,priority
0,1,31,36906466,1
1,1,279,3695190,1
2,1,398,497745,4
3,1,398,1133705,4
4,1,398,1139177,4
...,...,...,...,...
141206843,77257472,31,318,1
141206845,77257483,31,318,1
141206847,77257484,31,318,1
141206849,77257491,31,318,1


CPU times: user 2min 3s, sys: 10.1 s, total: 2min 13s
Wall time: 2min 13s


,source_item_id,target_item_id,weight
0,1,323,10
1,1,338,3
2,1,6999,3
3,1,11412,3
4,1,18343,3
...,...,...,...
95133823,77257472,318,8
95133824,77257483,318,8
95133825,77257484,318,8
95133826,77257491,318,8


In [7]:
id_counts_df = pd.read_csv("data/id_counts.csv").set_index('item_id')
display(id_counts_df)
item_views_dict = id_counts_df.views.to_dict()
item_counts_dict = id_counts_df.counts.to_dict()
print(item_views_dict[6199])
print(item_counts_dict[6199])

,page_id,title,views,counts
item_id,,,,
6199,12,Anarchism,31335,3540
38404,25,Autism,49693,2114
101038,39,Albedo,14573,2825
9659,290,A,25859,175
173,303,Alabama,52765,11125
...,...,...,...,...
76894635,62470350,Daming Zhu,16,0
76894633,62470423,Tony Dews,7,2
76896959,62470432,Samsung PL20,9,0


31335
3540


In [8]:
%%time
statements_sorted_df = statements_filtered_df.copy()
statements_sorted_df['target_counts'] = [item_counts_dict.get(i, 0) for i in statements_sorted_df.target_item_id]
statements_sorted_df['target_views'] = [item_views_dict.get(i, 0) for i in statements_sorted_df.target_item_id]
statements_sorted_df = statements_sorted_df.sort_values(by=['source_item_id', 'weight', 'target_counts', 'target_views'], \
                                                        ascending=[True, False, False, False]).reset_index(drop=True)
statements_sorted_df

CPU times: user 3min 20s, sys: 12.3 s, total: 3min 32s
Wall time: 3min 32s


,source_item_id,target_item_id,weight,target_counts,target_views
0,1,323,10,1183,56881
1,1,3695190,8,18,1148
2,1,36906466,8,0,0
3,1,273508,7,342,8005
4,1,837317,7,116,4778
...,...,...,...,...,...
95133823,77257472,318,8,1348,25105
95133824,77257483,318,8,1348,25105
95133825,77257484,318,8,1348,25105
95133826,77257491,318,8,1348,25105


In [9]:
# get unique entities and index of those entities
statements_np = statements_sorted_df[['source_item_id', 'target_item_id', 'weight']].to_numpy()
display(statements_np)

unique, indices = np.unique(statements_np[:,0], return_index=True)
print('source ids:', len(unique), unique)

indices = np.append(indices, len(statements_np))
print('id indices:', len(indices), indices[-10:])

array([[       1,      323,       10],
       [       1,  3695190,        8],
       [       1, 36906466,        8],
       ...,
       [77257484,      318,        8],
       [77257491,      318,        8],
       [77257493,      318,        8]])

source ids: 38488233 [       1        2        3 ... 77257484 77257491 77257493]
id indices: 38488234 [95133819 95133820 95133821 95133822 95133823 95133824 95133825 95133826
 95133827 95133828]


In [10]:
%%time
statements_groups = [[j, statements_np[indices[i]:indices[i+1], 1], statements_np[indices[i]:indices[i+1], 2]] for i, j in enumerate(unique)]
len(statements_groups)

CPU times: user 1min 53s, sys: 4.51 s, total: 1min 57s
Wall time: 1min 57s


38488233

In [11]:
%%time
grouped_df = pd.DataFrame(statements_groups, columns=['source_item_id', 'target_item_ids', 'weights'])
grouped_df

CPU times: user 25.6 s, sys: 1.29 s, total: 26.9 s
Wall time: 26.7 s


,source_item_id,target_item_ids,weights
0,1,"[323, 3695190, 36906466, 273508, 837317, 12086...","[10, 8, 8, 7, 7, 7, 7, 7, 7, 3, 3, 3, 3, 3, 3,..."
1,2,"[36133, 16502, 3504248, 323, 3400, 7879772, 18...","[8, 8, 8, 7, 7, 7, 7, 6, 3, 3, 3, 3, 3, 3, 3, ..."
2,3,"[483247, 203872, 937228, 1322005, 7239, 420, 1...","[8, 8, 8, 8, 7, 3, 3, 3, 3, 3, 3, 3, 3, 3]"
3,4,"[3, 2996394, 2956046, 3505845, 1931388, 149086...","[10, 8, 8, 8, 6, 3, 3, 3, 3, 3, 3, 3]"
4,5,"[215627, 15978631, 164509, 154954, 55983715, 1...","[11, 8, 8, 8, 8, 7, 6, 3, 3, 3, 3, 3, 3, 3, 3,..."
...,...,...,...
38488228,77257472,[318],[8]
38488229,77257483,[318],[8]
38488230,77257484,[318],[8]
38488231,77257491,[318],[8]


In [12]:
%%time
grouped_df.to_feather('data/graph.ftr')

CPU times: user 14 s, sys: 3.76 s, total: 17.8 s
Wall time: 31.8 s


### Format graph to run through node2vec and generate embeddings

In [32]:
%%time
statement_embed = statements_sorted_df[['source_item_id', 'target_item_id', 'weight']].copy()
statement_embed

CPU times: user 576 ms, sys: 586 ms, total: 1.16 s
Wall time: 1.15 s


,source_item_id,target_item_id,weight
0,1,323,10
1,1,3695190,8
2,1,36906466,8
3,1,273508,7
4,1,837317,7
...,...,...,...
95133823,77257472,318,8
95133824,77257483,318,8
95133825,77257484,318,8
95133826,77257491,318,8


In [33]:
weight_counts = pd.DataFrame(np.unique(statement_embed.weight.to_numpy(), return_counts=True)).T
weight_counts.sort_values(by=[1], ascending=False)

,0,1
0,3,28459978
3,8,28382430
1,6,18174689
2,7,17255232
11,16,1283620
...,...,...
59,99,1
48,64,1
57,94,1
49,66,1


In [34]:
id_counts_df.describe()

,page_id,views,counts
count,5.362174e+06,5.362174e+06,5.362174e+06
mean,2.691333e+07,4.554983e+02,2.272128e+01
std,1.872631e+07,6.783228e+03,3.564071e+02
min,1.200000e+01,5.000000e+00,0.000000e+00
25%,9.634382e+06,2.100000e+01,1.000000e+00
50%,2.543692e+07,5.300000e+01,3.000000e+00
75%,4.260177e+07,1.810000e+02,1.000000e+01
max,6.247333e+07,1.330002e+07,2.831270e+05


In [35]:
light_filter_ids = id_counts_df[(id_counts_df.counts > 0) & (id_counts_df.views > 5)].index
print(light_filter_ids)
heavy_filter_ids = id_counts_df[(id_counts_df.counts > 10) & (id_counts_df.views > 181)].index
print(heavy_filter_ids)

Int64Index([    6199,    38404,   101038,     9659,      173,    41746,
                  91,      868,   853997,   277751,
            ...
            56374723, 76892151, 15690957, 16696928, 76891681, 76891691,
            76894639, 76894633,  6034153, 21083961],
           dtype='int64', name='item_id', length=4475122)
Int64Index([    6199,    38404,   101038,     9659,      173,    41746,
                  91,      868,   853997,   277751,
            ...
            76149547,  1461197, 76373863, 76495389, 76431987,  1069320,
            76548457,  4339424,  9026959, 76874768],
           dtype='int64', name='item_id', length=847100)


In [36]:
%%time
statement_embed_f1 = statement_embed[statement_embed.source_item_id.isin(light_filter_ids) \
                                     & statement_embed.target_item_id.isin(light_filter_ids)]
statement_embed_f1 = pd.concat([statement_embed[statement_embed.weight > 8], \
                                statement_embed_f1[statement_embed_f1.weight <= 8]])
statement_embed_f1

CPU times: user 22.3 s, sys: 2 s, total: 24.3 s
Wall time: 24.3 s


,source_item_id,target_item_id,weight
0,1,323,10
80,4,3,10
92,5,215627,11
146,16,49,13
147,16,30,9
...,...,...,...
94942526,76952747,2526255,7
94942527,76952747,3288991,6
94952099,76961040,482994,8
94954142,76963213,5,8


In [37]:
weight_counts['f1'] = np.unique(statement_embed_f1.weight.to_numpy(), return_counts=True)[1]
weight_counts

,0,1,f1
0,3,28459978,4191500
1,6,18174689,4932208
2,7,17255232,6599007
3,8,28382430,4789447
4,9,218997,218997
...,...,...,...
76,231,1,1
77,294,1,1
78,357,1,1
79,441,1,1


In [38]:
%%time
statement_embed_f2 = statement_embed_f1[statement_embed_f1.source_item_id.isin(heavy_filter_ids) \
                                        & statement_embed_f1.target_item_id.isin(heavy_filter_ids)]
statement_embed_f2 = pd.concat([statement_embed_f1[statement_embed_f1.weight > 7], \
                                statement_embed_f2[statement_embed_f2.weight <= 7]])
statement_embed_f2

CPU times: user 6.06 s, sys: 492 ms, total: 6.56 s
Wall time: 6.55 s


,source_item_id,target_item_id,weight
0,1,323,10
80,4,3,10
92,5,215627,11
146,16,49,13
147,16,30,9
...,...,...,...
94672156,76568609,1860,3
94773913,76719589,1484397,6
94792939,76743491,214548,7
94792940,76743491,1860,3


In [40]:
weight_counts['f2'] = np.unique(statement_embed_f2.weight.to_numpy(), return_counts=True)[1]
display(weight_counts)
weight_counts.sum()

,0,1,f1,f2
0,3,28459978,4191500,1008991
1,6,18174689,4932208,1065829
2,7,17255232,6599007,1741267
3,8,28382430,4789447,4789447
4,9,218997,218997,218997
...,...,...,...,...
76,231,1,1,1
77,294,1,1,1
78,357,1,1,1
79,441,1,1,1


0         6571
1     95133828
f1    23373661
f2    11467033
dtype: int64

In [41]:
%%time
#Exports to the format used by node2vec
statement_embed_f1.to_csv('data/statements_embed_large.csv', header=False, index=False, sep=' ')
statement_embed_f2.to_csv('data/statements_embed_small.csv', header=False, index=False, sep=' ')

CPU times: user 1min, sys: 583 ms, total: 1min
Wall time: 1min 1s
